In [ ]:
import os
import ase
import numpy as np
import matplotlib.pyplot as plt
from ase.build import fcc100,add_adsorbate
from ase import Atoms
import ase.visualize as asevisual
from ase import data as asedata

from ase.calculators.eam import EAM
from ase.optimize import FIRE, LBFGS
from ase.neb import NEB
from ase.neb import NEBTools
from ase.io import Trajectory
from ase.vibrations import Vibrations

<h1> Demonstration<h1>
<h5>Useful page: https://wiki.fysik.dtu.dk/ase/ase/neb.html<h5>
<h2>Direct hopping of adatom on silver fcc(100) surface.<h2>
<h2>Step 1: Prepare initial and final state structures<h2>
<h3>https://wiki.fysik.dtu.dk/ase/ase/build/surface.html<h3>

In [ ]:
# Trying to make a folder for exporting files
try:    
    os.mkdir('demo_dump/')
except:
    pass
# ASE build in function to generate surface
fcc100_Pd = fcc100('Pd', (6,6,4), a=None, vacuum=10, orthogonal=True, periodic=True)
# Get the NN separation between Ag-Ag
NN_dist= asedata.reference_states[46]['a']/np.sqrt(2)

# Get a location of the adatom to put
adatom_loc_xy = fcc100_Pd.positions[fcc100_Pd.positions[:,2].argsort()[-1]]+ NN_dist*np.array([-3.5,-3.5,0])
# This is relative to the 'top' of the slab
adatom_loc_height= NN_dist/np.sqrt(2)
# Make an ase obj for the initial state
ini_ase=fcc100_Pd.copy()
add_adsorbate(ini_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy[0], adatom_loc_xy[1]))

# Similrar to above, but with the adatom shifted to the next holo-site
adatom_loc_xy = adatom_loc_xy+NN_dist*np.array([1,0,0])
fin_ase=fcc100_Pd.copy()
add_adsorbate(fin_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy[0], adatom_loc_xy[1]))

# Sanity check if the structure is okay
# https://wiki.fysik.dtu.dk/ase/ase/visualize/visualize.html#module-ase.visualize
asevisual.view([ini_ase,fin_ase])

<h2>Step 2: Optimize the structures!<h2>

In [ ]:
# For newbies using ASE, the documentation of Atoms object in ASE:
# https://wiki.fysik.dtu.dk/ase/ase/atoms.html
# (Read it when you need it)

# Set a calculator for each structure, here we will stick to simple classical force field (EAM)
ini_ase.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
ini_e_before= ini_ase.get_potential_energy()
# Setup the optimization algorithm, here we use FIRE/LBFGS
# Choose the one suits your system
# https://wiki.fysik.dtu.dk/ase/ase/optimize.html
# dyn = FIRE(ini_ase)
dyn = LBFGS(ini_ase)
# Execute the optimization, with a threshold of fmax=0.05
# The convergence criterion is that the force on all individual atoms should be less than fmax
dyn.run(fmax=0.05)
ini_e_after= ini_ase.get_potential_energy()
print(f'Structure Optimized Energy {ini_e_before} -> {ini_e_after}')

# Repeat for final state
fin_ase.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
fin_e_before= fin_ase.get_potential_energy()
# dyn = FIRE(fin_ase)
dyn = LBFGS(fin_ase)
dyn.run(fmax=0.05)
fin_e_after= fin_ase.get_potential_energy()
print(f'Structure Optimized Energy {fin_e_before} -> {fin_e_after}')

<h2>Step 3: ci-NEB Time. Initalize the states of the band<h2>

In [ ]:
# Make a list for images of the band, the first one is the initial, last one is final
# The middle ones are initalized as a copy of the initial state (will be adjusted afterwards)
# Let's put 7 images in the band
Nimages = 7
# From ASE documentation
# https://wiki.fysik.dtu.dk/ase/ase/neb.html
# Be sure to use the copy method (or similar) to create new instances of atoms within the list of images fed to the NEB.
# Do not use something like [initial for i in range(3)], as it will only create references to the original atoms object.
NEB_images = [ini_ase.copy() for item in np.arange(Nimages+1)]
NEB_images+= [fin_ase]
# Intiallize the ciNEB object
# climb=True for climing image
# improved tangent: 
# H. Jonsson, G. Mills, and K. W. Jacobsen, in ‘Classical and Quantum Dynamics in Condensed Phase Systems’, edited by B. J. Berne, G. Cicotti, and D. F. Coker, World Scientific, 1998
neb=NEB(NEB_images, k=0.1, climb=True, method='improvedtangent')
# Adjust the images to an interpolation of positions between inital and final states
# IDPP: ‘Improved initial guess for minimum energy path calculations.’, S. Smidstrup, A. Pedersen, K. Stokbro and H. Jonsson, J. Chem. Phys. 140, 214106 (2014)
neb.interpolate('idpp',mic=True)
# Each bead has its own ASE calculator
for bead in NEB_images:
    bead.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
# Take a look at it first
asevisual.view(NEB_images)

<h2>Step 4: Really ci-NEB Time<h2>

In [ ]:
# Make a trajectory file, contains the images of each NEB iteration
# dyn=FIRE(neb, trajectory='demo_dump/NEB.traj', logfile='demo_dump/neb.log')
dyn=LBFGS(neb, trajectory='demo_dump/NEB.traj', logfile='demo_dump/neb.log')

# Make a trajectory file for each bead on the band for each NEB iteration
for bead_id in range(1,Nimages+1):
    traj = Trajectory('demo_dump/neb-%d.traj' % bead_id, 'w', NEB_images[bead_id])
    dyn.attach(traj)
# Run ciNEB
dyn.run(fmax=0.05)

<h2>Step 5: Extracting Stuffs<h2>

In [ ]:
# This gives the forward barrier
# just the energy difference of transition state and initial
nebtools = NEBTools(NEB_images)
barrier= nebtools.get_barrier(fit=False)
print(f'The barrier of forward reaction is {barrier[0]}eV')

In [ ]:
# Alternate way, calculate by yourself
nebtools = NEBTools(NEB_images)
IS_E= NEB_images[0].get_potential_energy()
# TS_E= nebtools.get_barrier(fit=False,raw=True)[0]
# or
TS_E= NEB_images[4].get_potential_energy()
FS_E= NEB_images[-1].get_potential_energy()
# Why is it [0],[4],[-1]???

Eact_IT = TS_E-IS_E
Eact_FT = TS_E-FS_E
print(f'The barrier of forward reaction is {Eact_IT:.4f} eV')
print(f'The barrier of backward reaction is {Eact_FT:.4f} eV')

In [ ]:
# These are for plotting in matplotlib
fig = plt.figure(dpi=300)
ax = fig.add_axes((0.15, 0.15, 0.8, 0.75))
nebtools.plot_band(ax)
plt.show()
plt.close()

In [ ]:
# You might have to first remove the old output files before you run this
# Planck constant
h = 4.135667689e-15
# No one wants to solve the whole Hessian involving all atoms (most of them are not moving)
# This is the ID of the atom which you want to include when solving the vibrational energy
indices = [144]
# https://wiki.fysik.dtu.dk/ase/ase/vibrations/modes.html
# Solve the vibrational energy for ini state
vib_IS = Vibrations(NEB_images[0],indices=indices,name="demo_dump/vib_IS")
vib_IS.clean()
vib_IS.run()
# Get the vibrational energies
IS_energies = vib_IS.get_energies()
# Get a product of them (for calculation of prefactor with hTST)
IS_real = [i.real for i in IS_energies if i.real != 0]

# Solve the vibrational energy for transition state
vib_TS = Vibrations(NEB_images[4],indices=indices,name="demo_dump/vib_TS")
vib_TS.clean()
vib_TS.run()
# Get the vibrational energies
TS_energies = vib_TS.get_energies()
# Get a product of them (for calculation of prefactor with hTST)
# One of them is supposed to be imaginary (is it?)
TS_real = [i.real for i in TS_energies if i.real != 0]

# Solve the vibrational energy for fin state
vib_FS = Vibrations(NEB_images[-1],indices=indices,name="demo_dump/vib_FS")
vib_FS.clean()
vib_FS.run()
FS_energies = vib_FS.get_energies()
FS_real = [i.real for i in FS_energies if i.real != 0]

nu_IT = (np.prod(IS_real)/np.prod(TS_real))/h
nu_FT = (np.prod(FS_real)/np.prod(TS_real))/h
print(f'The prefactor of forward reaction is {nu_IT:.4e} Hz')
print(f'The prefactor of backward reaction is {nu_FT:.4e} Hz')

<h1>Your turn!<h1>

<h2> Ex.1: Exchange Diffusion Mechanism on FCC(100) <h2>

In [ ]:
import os
try:    
    os.mkdir('ex1_dump/')
except:
    pass
# ASE build in function to generate surface
fcc100_Pd = fcc100('Pd', (6,6,4), a=None, vacuum=10, orthogonal=True, periodic=True)
# Get the NN separation between Ag-Ag
NN_dist= asedata.reference_states[46]['a']/np.sqrt(2)

# Get a location of the adatom to put
adatom_loc_xy = fcc100_Pd.positions[fcc100_Pd.positions[:,2].argsort()[-1]]+ NN_dist*np.array([-3.5,-3.5,0])
# This is relative to the 'top' of the slab
adatom_loc_height= NN_dist/np.sqrt(2)
# Make an ase obj for the initial state
ini_ase=fcc100_Pd.copy()
add_adsorbate(ini_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy[0], adatom_loc_xy[1]))

# Similrar to above, but with the adatom shifted to the next holo-site
adatom_loc_xy = adatom_loc_xy+NN_dist*np.array([1,1,0])
fin_ase=fcc100_Pd.copy()
add_adsorbate(fin_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy[0], adatom_loc_xy[1]))
fin_pos=fin_ase.positions.copy()
fin_pos[144] = fin_ase.positions[122]
fin_pos[122] = fin_ase.positions[144]
fin_ase.set_positions(fin_pos)
# Sanity check if the structure is okay
asevisual.view([ini_ase,fin_ase])

In [ ]:
ini_ase.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
ini_e_before= ini_ase.get_potential_energy()
# Setup the optimization algorithm, here we use FIRE/LBFGS
# Choose the one suits your system
# https://wiki.fysik.dtu.dk/ase/ase/optimize.html
# dyn = FIRE(ini_ase)
dyn = LBFGS(ini_ase)
dyn.run(fmax=0.05)
ini_e_after= ini_ase.get_potential_energy()
print(f'Structure Optimized Energy {ini_e_before} -> {ini_e_after}')

fin_ase.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
fin_e_before= fin_ase.get_potential_energy()
# dyn = FIRE(fin_ase)
dyn = LBFGS(fin_ase)
dyn.run(fmax=0.05)
fin_e_after= fin_ase.get_potential_energy()
print(f'Structure Optimized Energy {fin_e_before} -> {fin_e_after}')

In [ ]:
# Make a list for images of the band, the first one is the initial, last one is final
# The middle ones are initalized as a copy of the initial state (will be adjusted afterwards)
# Let's put 7 images in the band
Nimages = 7
# From ASE documentation
# Be sure to use the copy method (or similar) to create new instances of atoms within the list of images fed to the NEB.
# Do not use something like [initial for i in range(3)], as it will only create references to the original atoms object.
NEB_images = [ini_ase.copy() for item in np.arange(Nimages+1)]
NEB_images+= [fin_ase]
# Intiallize the NEB object
neb=NEB(NEB_images, k=0.1, climb=True, method='improvedtangent')
# Adjust the images to an interpolation of positions between inital and final states
# IDPP: ‘Improved initial guess for minimum energy path calculations.’, S. Smidstrup, A. Pedersen, K. Stokbro and H. Jonsson, J. Chem. Phys. 140, 214106 (2014)
neb.interpolate('idpp',mic=True)
# Each bead has its own ASE calculator
for bead in NEB_images:
    bead.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
# Take a look at it first
asevisual.view(NEB_images)

In [ ]:
# Make a trajectory file, contains the images of each NEB iteration
# Setup the optimization algorithm, here we use FIRE/LBFGS
# Choose the one suits your system
# https://wiki.fysik.dtu.dk/ase/ase/optimize.html
# dyn=FIRE(neb, trajectory='ex1_dump/NEB.traj', logfile='ex1_dump/neb.log')
dyn=LBFGS(neb, trajectory='ex1_dump/NEB.traj', logfile='ex1_dump/neb.log')
# Make a trajectory file for each bead on the band for each NEB iteration
for bead_id in range(1,Nimages+1):
    traj = Trajectory('ex1_dump/neb-%d.traj' % bead_id, 'w', NEB_images[bead_id])
    dyn.attach(traj)
# Run ciNEB
dyn.run(fmax=0.05)

In [ ]:
# Alternate way, calculate by yourself
nebtools = NEBTools(NEB_images)
IS_E= NEB_images[0].get_potential_energy()
# TS_E= nebtools.get_barrier(fit=False,raw=True)[0]
# or
TS_E= NEB_images[4].get_potential_energy()
FS_E= NEB_images[-1].get_potential_energy()
# Why is it [0],[4],[-1]???

Eact_IT = TS_E-IS_E
Eact_FT = TS_E-FS_E
print(f'The barrier of forward reaction is {Eact_IT:.4f} eV')
print(f'The barrier of backward reaction is {Eact_FT:.4f} eV')

In [ ]:
fig = plt.figure(dpi=300)
ax = fig.add_axes((0.15, 0.15, 0.8, 0.75))
nebtools.plot_band(ax)
plt.show()
plt.close()

In [ ]:
# You might have to first remove the old output files before you run this
h = 4.135667689e-15
# No one wants to solve the whole Hessian involving all atoms (most of them are not moving)
indices = [116,121,122,123,128,144]
# Solve the vibrational energy for ini state
vib_IS = Vibrations(NEB_images[0],indices=indices,name="ex1_dump/vib_IS")
vib_IS.clean()
vib_IS.run()
# Get the vibrational energies
IS_energies = vib_IS.get_energies()
# Get a product of them (for calculation of prefactor with hTST)
IS_real = [i.real for i in IS_energies if i.real != 0]

# Solve the vibrational energy for transition state
vib_TS = Vibrations(NEB_images[4],indices=indices,name="ex1_dump/vib_TS")
vib_TS.clean()
vib_TS.run()
# Get the vibrational energies
TS_energies = vib_TS.get_energies()
# Get a product of them (for calculation of prefactor with hTST)
# One of them is supposed to be imaginary (is it?)
TS_real = [i.real for i in TS_energies if i.real != 0]

# Solve the vibrational energy for fin state
vib_FS = Vibrations(NEB_images[-1],indices=indices,name="ex1_dump/vib_FS")
vib_FS.clean()
vib_FS.run()
FS_energies = vib_FS.get_energies()
FS_real = [i.real for i in FS_energies if i.real != 0]

nu_IT = (np.prod(IS_real)/np.prod(TS_real))/h
nu_FT = (np.prod(FS_real)/np.prod(TS_real))/h
print(f'The prefactor of forward reaction is {nu_IT:.4e} Hz')
print(f'The prefactor of backward reaction is {nu_FT:.4e} Hz')

<h2> Ex.2: Bond Breaking Direct Hop <h2>

In [ ]:
import os
try:    
    os.mkdir('ex2_dump/')
except:
    pass
# ASE build in function to generate surface
fcc100_Pd = fcc100('Pd', (6,6,4), a=None, vacuum=10, orthogonal=True, periodic=True)
# Get the NN separation between Ag-Ag
NN_dist= asedata.reference_states[46]['a']/np.sqrt(2)

# Get a location of the adatom to put
adatom_loc_xy = fcc100_Pd.positions[fcc100_Pd.positions[:,2].argsort()[-1]]+ NN_dist*np.array([-3.5,-3.5,0])
adatom_loc_xy_2 = fcc100_Pd.positions[fcc100_Pd.positions[:,2].argsort()[-1]]+ NN_dist*np.array([-2.5,-3.5,0])
# This is relative to the 'top' of the slab
adatom_loc_height= NN_dist/np.sqrt(2)
# Make an ase obj for the initial state
ini_ase=fcc100_Pd.copy()
add_adsorbate(ini_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy[0], adatom_loc_xy[1]))
add_adsorbate(ini_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy_2[0], adatom_loc_xy_2[1]))

# Similrar to above, but with the adatom shifted to the next holo-site
adatom_loc_xy_2 = adatom_loc_xy_2+NN_dist*np.array([1,0,0])
fin_ase=fcc100_Pd.copy()
add_adsorbate(fin_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy[0], adatom_loc_xy[1]))
add_adsorbate(fin_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy_2[0], adatom_loc_xy_2[1]))
# Sanity check if the structure is okay
asevisual.view([ini_ase,fin_ase])

In [ ]:
ini_ase.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
ini_e_before= ini_ase.get_potential_energy()
dyn = LBFGS(ini_ase)
# dyn = FIRE(ini_ase)
dyn.run(fmax=0.05)
ini_e_after= ini_ase.get_potential_energy()
print(f'Structure Optimized Energy {ini_e_before} -> {ini_e_after}')

fin_ase.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
fin_e_before= fin_ase.get_potential_energy()
# dyn = FIRE(fin_ase)
dyn = LBFGS(fin_ase)
dyn.run(fmax=0.05)
fin_e_after= fin_ase.get_potential_energy()
print(f'Structure Optimized Energy {fin_e_before} -> {fin_e_after}')

In [ ]:
# Make a list for images of the band, the first one is the initial, last one is final
# The middle ones are initalized as a copy of the initial state (will be adjusted afterwards)
# Let's put 7 images in the band
Nimages = 7
# From ASE documentation
# Be sure to use the copy method (or similar) to create new instances of atoms within the list of images fed to the NEB.
# Do not use something like [initial for i in range(3)], as it will only create references to the original atoms object.
NEB_images = [ini_ase.copy() for item in np.arange(Nimages+1)]
NEB_images+= [fin_ase]
# Intiallize the NEB object
neb=NEB(NEB_images, k=0.1, climb=True, method='improvedtangent')
# Adjust the images to an interpolation of positions between inital and final states
# IDPP: ‘Improved initial guess for minimum energy path calculations.’, S. Smidstrup, A. Pedersen, K. Stokbro and H. Jonsson, J. Chem. Phys. 140, 214106 (2014)
neb.interpolate('idpp',mic=True)
# Each bead has its own ASE calculator
for bead in NEB_images:
    bead.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
# Take a look at it first
asevisual.view(NEB_images)

In [ ]:
# Make a trajectory file, contains the images of each NEB iteration
# Also set up the optimization algorithm for NEB, here we have FIRE or LBFGS
# https://wiki.fysik.dtu.dk/ase/ase/optimize.html
# dyn=FIRE(neb, trajectory='ex2_dump/NEB.traj', logfile='ex2_dump/neb.log')
# or
dyn=LBFGS(neb, trajectory='ex2_dump/NEB.traj', logfile='ex2_dump/neb.log')
# Make a trajectory file for each bead on the band for each NEB iteration
for bead_id in range(1,Nimages+1):
    traj = Trajectory('ex2_dump/neb-%d.traj' % bead_id, 'w', NEB_images[bead_id])
    dyn.attach(traj)
# Run ciNEB
dyn.run(fmax=0.05)

In [ ]:
# Alternate way, calculate by yourself
nebtools = NEBTools(NEB_images)
IS_E= NEB_images[0].get_potential_energy()
# TS_E= nebtools.get_barrier(fit=False,raw=True)[0]
# or
TS_E= NEB_images[4].get_potential_energy()
FS_E= NEB_images[-1].get_potential_energy()
# Why is it [0],[4],[-1]???

Eact_IT = TS_E-IS_E
Eact_FT = TS_E-FS_E
print(f'The barrier of forward reaction is {Eact_IT:.4f} eV')
print(f'The barrier of backward reaction is {Eact_FT:.4f} eV')

In [ ]:
fig = plt.figure(dpi=300)
ax = fig.add_axes((0.15, 0.15, 0.8, 0.75))
nebtools.plot_band(ax)
plt.show()
plt.close()

In [ ]:
# You might have to first remove the old output files before you run this
h = 4.135667689e-15
# No one wants to solve the whole Hessian involving all atoms (most of them are not moving)
indices = [144,145]
# Solve the vibrational energy for ini state
vib_IS = Vibrations(NEB_images[0],indices=indices,name="ex2_dump/vib_IS")
vib_IS.clean()
vib_IS.run()
# Get the vibrational energies
IS_energies = vib_IS.get_energies()
# Get a product of them (for calculation of prefactor with hTST)
IS_real = [i.real for i in IS_energies if i.real != 0]

# Solve the vibrational energy for transition state
vib_TS = Vibrations(NEB_images[4],indices=indices,name="ex2_dump/vib_TS")
vib_TS.clean()
vib_TS.run()
# Get the vibrational energies
TS_energies = vib_TS.get_energies()
# Get a product of them (for calculation of prefactor with hTST)
# One of them is supposed to be imaginary (is it?)
TS_real = [i.real for i in TS_energies if i.real != 0]

# Solve the vibrational energy for fin state
vib_FS = Vibrations(NEB_images[-1],indices=indices,name="ex2_dump/vib_FS")
vib_FS.clean()
vib_FS.run()
FS_energies = vib_FS.get_energies()
FS_real = [i.real for i in FS_energies if i.real != 0]

nu_IT = (np.prod(IS_real)/np.prod(TS_real))/h
nu_FT = (np.prod(FS_real)/np.prod(TS_real))/h
print(f'The prefactor of forward reaction is {nu_IT:.4e} Hz')
print(f'The prefactor of backward reaction is {nu_FT:.4e} Hz')

<h2> Ex.3: Hmmm <h2>

In [ ]:
import os
try:    
    os.mkdir('ex3_dump/')
except:
    pass
# ASE build in function to generate surface
fcc100_Pd = fcc100('Pd', (6,6,4), a=None, vacuum=10, orthogonal=True, periodic=True)
# Get the NN separation between Ag-Ag
NN_dist= asedata.reference_states[46]['a']/np.sqrt(2)

# Get a location of the adatom to put
adatom_loc_xy = fcc100_Pd.positions[fcc100_Pd.positions[:,2].argsort()[-1]]+ NN_dist*np.array([-3.5,-3.5,0])
# This is relative to the 'top' of the slab
adatom_loc_height= NN_dist/np.sqrt(2)
# Make an ase obj for the initial state
ini_ase=fcc100_Pd.copy()
add_adsorbate(ini_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy[0], adatom_loc_xy[1]))

# Similrar to above, but with the adatom shifted to the next holo-site
adatom_loc_xy = adatom_loc_xy+NN_dist*np.array([1,1,0])
fin_ase=fcc100_Pd.copy()
add_adsorbate(fin_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy[0], adatom_loc_xy[1]))
# Sanity check if the structure is okay
asevisual.view([ini_ase,fin_ase])

In [ ]:
ini_ase.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
ini_e_before= ini_ase.get_potential_energy()
# dyn = FIRE(ini_ase)
dyn = LBFGS(ini_ase)
dyn.run(fmax=0.05)
ini_e_after= ini_ase.get_potential_energy()
print(f'Structure Optimized Energy {ini_e_before} -> {ini_e_after}')

fin_ase.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
fin_e_before= fin_ase.get_potential_energy()
# dyn = FIRE(fin_ase)
dyn = LBFGS(fin_ase)
dyn.run(fmax=0.05)
fin_e_after= fin_ase.get_potential_energy()
print(f'Structure Optimized Energy {fin_e_before} -> {fin_e_after}')

In [ ]:
# Make a list for images of the band, the first one is the initial, last one is final
# The middle ones are initalized as a copy of the initial state (will be adjusted afterwards)
# Let's put 7 images in the band
Nimages = 7
# From ASE documentation
# Be sure to use the copy method (or similar) to create new instances of atoms within the list of images fed to the NEB.
# Do not use something like [initial for i in range(3)], as it will only create references to the original atoms object.
NEB_images = [ini_ase.copy() for item in np.arange(Nimages+1)]
NEB_images+= [fin_ase]
# Intiallize the NEB object
neb=NEB(NEB_images, k=0.1, climb=True, method='improvedtangent')
# Adjust the images to an interpolation of positions between inital and final states
# IDPP: ‘Improved initial guess for minimum energy path calculations.’, S. Smidstrup, A. Pedersen, K. Stokbro and H. Jonsson, J. Chem. Phys. 140, 214106 (2014)
neb.interpolate('idpp',mic=True)
# Each bead has its own ASE calculator
for bead in NEB_images:
    bead.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
# Take a look at it first
asevisual.view(NEB_images)

In [ ]:
# Make a trajectory file, contains the images of each NEB iteration
# dyn=FIRE(neb, trajectory='ex3_dump/NEB.traj', logfile='ex3_dump/neb.log')
dyn=LBFGS(neb, trajectory='ex3_dump/NEB.traj', logfile='ex3_dump/neb.log')
# Make a trajectory file for each bead on the band for each NEB iteration
for bead_id in range(1,Nimages+1):
    traj = Trajectory('ex3_dump/neb-%d.traj' % bead_id, 'w', NEB_images[bead_id])
    dyn.attach(traj)
# Run ciNEB
dyn.run(fmax=0.05)

In [ ]:
# Alternate way, calculate by yourself
nebtools = NEBTools(NEB_images)
IS_E= NEB_images[0].get_potential_energy()
# TS_E= nebtools.get_barrier(fit=False,raw=True)[0]
# or
TS_E= NEB_images[4].get_potential_energy()
FS_E= NEB_images[-1].get_potential_energy()
# Why is it [0],[4],[-1]???
print(f'The barrier of forward reaction is {Eact_IT:.4f} eV')
print(f'The barrier of backward reaction is {Eact_FT:.4f} eV')

In [ ]:
fig = plt.figure(dpi=300)
ax = fig.add_axes((0.15, 0.15, 0.8, 0.75))
nebtools.plot_band(ax)
plt.show()
plt.close()

In [ ]:
# You might have to first remove the old output files before you run this
h = 4.135667689e-15
# No one wants to solve the whole Hessian involving all atoms (most of them are not moving)
indices = [144]
# Solve the vibrational energy for ini state
vib_IS = Vibrations(NEB_images[0],indices=indices,name="ex3_dump/vib_IS")
vib_IS.clean()
vib_IS.run()
# Get the vibrational energies
IS_energies = vib_IS.get_energies()
# Get a product of them (for calculation of prefactor with hTST)
IS_real = [i.real for i in IS_energies if i.real != 0]

# Solve the vibrational energy for transition state
vib_TS = Vibrations(NEB_images[4],indices=indices,name="ex3_dump/vib_TS")
vib_TS.clean()
vib_TS.run()
# Get the vibrational energies
TS_energies = vib_TS.get_energies()
# Get a product of them (for calculation of prefactor with hTST)
# One of them is supposed to be imaginary (is it?)
TS_real = [i.real for i in TS_energies if i.real != 0]

# Solve the vibrational energy for fin state
vib_FS = Vibrations(NEB_images[-1],indices=indices,name="ex3_dump/vib_FS")
vib_FS.clean()
vib_FS.run()
FS_energies = vib_FS.get_energies()
FS_real = [i.real for i in FS_energies if i.real != 0]

nu_IT = (np.prod(IS_real)/np.prod(TS_real))/h
nu_FT = (np.prod(FS_real)/np.prod(TS_real))/h
print(f'The prefactor of forward reaction is {nu_IT:.4e} Hz')
print(f'The prefactor of backward reaction is {nu_FT:.4e} Hz')

<h1>But hold on again!!!<h1>
<h2>If you take a look at the MEP (view the NEH.traj)<h2>
<h2>This doesn't look like a physically feasible process!<h2>
<h2>You should instead break it down into multiple processes, do a series of NEB on it<h2>

================================================================================================================================================
<h2>Bonus Materials<h2>

<h2>If you are not familiar with ASE, the Atoms object is the class you have to handle most of them time<h2>
https://wiki.fysik.dtu.dk/ase/gettingstarted/tut01_molecule/molecule.html

You can modify the positions of atoms, add/remove atoms like we have done in the examples above

Please go ahead to explore the way this is handled

<h2>You may need this in other tutorials<h2>
<h2>In-/ Outputting structures (ASE)<h2>
https://wiki.fysik.dtu.dk/ase/ase/io/io.html

In [ ]:
from ase import io as aseio
# Export
aseio.write('demo_dump/export_ini.con',ini_ase, parallel=False, append=False)
aseio.write('demo_dump/export_fin.con',fin_ase, parallel=False, append=False)

<h2>So now you want look at the trajectories generated<h2>
<h2>e.g. the NEB.traj we generated in the demonstration what should you do?<h2>
<h2>1) Load the structure<h2>
<h2>2) Now you have a list of Atoms (or "trajectory" in ASE terms)<h2>
<h2>3) view them as we did before<h2>

In [ ]:
trajectory_input= aseio.read('demo_dump/NEB.traj',index=':',parallel=False)
asevisual.view(trajectory_input)

<h2>The index here is ":", which means read all structures in that file (it contains all images states in all iterations!)<h2>
<h2>If you just want to see the last 9<h2>

In [ ]:
trajectory_input= aseio.read('demo_dump/NEB.traj',index='-9:',parallel=False)
asevisual.view(trajectory_input)